# Measuring Neighborhood Availability of Alcohol

Use data from KY’s Department of Alcoholic Beverage Control (ABC) to calculate the availability of alcohol in Fayette county’s neighborhoods.  In particular for each neighborhood, calculate the rate of liquor licenses per capita (the number of licenses divided by the neighborhood’s population).  Show the top 20 neighborhoods with the highest rate of alcohol availability. Show the top 20 neighborhoods with the highest number of licenses.  Discuss whether or not these two top-20 lists differ and how.  Define neighborhood as a US Census Bureau’s tract (discussed below).  Present this as a notebook, complete with documentation and code. Use the notebook to communicate about the data, process, and results.

***The first step in this project will be to load the liquor license data into a pandas dataframe (Liquor license data was supplied for the project, and orginally taken from https://abc.ky.gov/). This dataframe contains all liquor licenses issued in Fayette County.***



In [1]:
import pandas as pd
import numpy as np


In [6]:
# use selenium to scrape website and get latest list of alcohol licenses


In [16]:
# Get alcohol license data...

# path of Fayette County liquor licenses file
license_excel = r'/Users/Matthew/Desktop/Licenses.xlsx'

# create dataframe from Fayette County liquor license file
df_license = pd.read_csv(license_excel, parse_dates=['IssueDate','EffectiveDate','ExpiryDate'])


# insert unique values for missing SiteID records; otherwise batch census tract extraction through 
# census api deletes records without unique id
#df_license.loc[df_license['SiteID'].isna(), 'SiteID'] = ['missing_' + str(i+1) for i in \ 
 #range(df_license['SiteID'].isna().sum())]


# casting SiteID variable to string in order to merge properly in steps below
#df_license['SiteID'] = df_license['SiteID'].astype('string')



# display first 5 rows of dataframe
df_license.head()

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 10-12: invalid continuation byte

## ***Prepare the dataframe. We will use the Census Bureau API for geocoding addresses and extracting Census Tracts for each liquor license location.***

In [158]:
# creating columns for city, state, and zip code to use for batch census tract extraction through census api; values filled with nan
df_license[['api_city', 'api_state', 'api_zip']] = np.nan

# split string for PremisesCityState into two columns; api_city is finalized; api_state needs another step
df_license[['api_city', 'api_state']] = df_license['PremisesCityState'].str.split(', ', expand=True)

# modify api_state by splitting string value into state and zip code into two columns
df_license[['api_state', 'api_zip']] = df_license['api_state'].str.split(' ', expand=True)

# clean up any leading & trailing whitespace
df_license['PremisesStreet'] = df_license['PremisesStreet'].str.strip()
df_license['api_city'] = df_license['api_city'].str.strip()
df_license['api_state'] = df_license['api_state'].str.strip()
df_license['api_zip'] = df_license['api_zip'].str.strip()

# Print total liquor licenses in Fayette County (rows in dataframe)
print('Total liquor licenses in Fayette County (modified dataframe): ', len(df_license))

# display first 5 rows of dataframe
df_license.head()

Total liquor licenses in Fayette County (modified dataframe):  1807


,SiteID,County,City,DBA,Licensee,PremisesStreet,PremisesCityState,LicenseNumber,LicenseType,Status,IssueDate,EffectiveDate,ExpiryDate,api_city,api_state,api_zip
0,missing_1,Fayette,Lexington,2022 Camp Out,"HORSIN' AROUND CAMP, INC.",1540 University Dr,"Lexington, KY 40506",034-TA-192962,Special Temporary Alcoholic Beverage Auction L...,Active,9/26/22,10/1/22,10/20/22,Lexington,KY,40506
1,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,"Lexington, KY 40507",034-NQ2-2920,NQ2 Retail Drink License,Active,1/29/16,1/29/16,11/30/22,Lexington,KY,40507
2,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,"Lexington, KY 40507",034-SP-1576,Sampling License,Active,1/29/16,1/29/16,11/30/22,Lexington,KY,40507
3,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,"Lexington, KY 40507",034-HI-29,Hotel In-Room License,Active,1/29/16,1/29/16,11/30/22,Lexington,KY,40507
4,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,"Lexington, KY 40507",034-SB-1582,Supplemental Bar License,Active,1/29/16,1/29/16,11/30/22,Lexington,KY,40507


## ***Create new dataframe. Many locations have multiple liquor licenses, so we will create a new dataframe for unique locations to further assess liquor availability.***

In [159]:
# create new dataframe for unique locations; remove duplicate locations
df_location = df_license.drop_duplicates(subset='SiteID')

# reset index after dropping duplicate locations
df_location.reset_index(inplace=True, drop=True)

# report number of unique locations of liquor availability in Fayette County
print('Total unique locations to purchase alcohol in Fayette County: ', len(df_location))

# display first 5 rows of dataframe
df_location.head()

Total unique locations to purchase alcohol in Fayette County:  818


,SiteID,County,City,DBA,Licensee,PremisesStreet,PremisesCityState,LicenseNumber,LicenseType,Status,IssueDate,EffectiveDate,ExpiryDate,api_city,api_state,api_zip
0,missing_1,Fayette,Lexington,2022 Camp Out,"HORSIN' AROUND CAMP, INC.",1540 University Dr,"Lexington, KY 40506",034-TA-192962,Special Temporary Alcoholic Beverage Auction L...,Active,9/26/22,10/1/22,10/20/22,Lexington,KY,40506
1,22790.0,Fayette,Lexington,21c Museum Hotel Lexington,167 MAIN HOTEL LLC,167 W Main St,"Lexington, KY 40507",034-NQ2-2920,NQ2 Retail Drink License,Active,1/29/16,1/29/16,11/30/22,Lexington,KY,40507
2,26561.0,Fayette,Lexington,A Cup of Common Wealth,A Cup of Common Wealth LLC,105 Eastern Ave,"Lexington, KY 40508",034-NQ4-191200,NQ4 Retail Malt Beverage Drink License,Active,6/28/22,6/28/22,11/30/22,Lexington,KY,40508
3,5119.0,Fayette,Lexington,A P Suggins Bar And Grill,"ROMANY RESTAURANTS, INC.",345 Romany Rd,"Lexington, KY 40502",034-RS-2332,Special Sunday Retail Drink License,Active,5/13/13,6/25/13,8/31/23,Lexington,KY,40502
4,28033.0,Fayette,Lexington,Agave & Rye Epic Tacos,Agave & Rye Lexington LLC,3535 Nicholasville Rd,"Lexington, KY 40503",034-NQ2-3989,NQ2 Retail Drink License,Active,6/17/19,6/17/19,11/30/22,Lexington,KY,40503


## ***Use the Census Bureau API for geocoding locations and extracting tracts for each location.***

In [134]:
# write csv file according to specs for batch census tract extraction through api; saving to google drive; no headers nor index from dataframe in csv
df_license[['SiteID', 'PremisesStreet', 'api_city', 'api_state', 'api_zip']].to_csv(r'/content/drive/MyDrive/Classes/2022_Fall_DS501/batch_license.csv', header=False, index=False)

In [124]:
# terminal command to use batch_license file from above to get tract data; save as geocoderesult_location.csv in google drive
%%bash
curl --form addressFile=@/content/drive/MyDrive/Classes/2022_Fall_DS501/batch_license.csv --form benchmark=Public_AR_Census2020 --form vintage=Census2020_Census2020 --form layers=6 https://geocoding.geo.census.gov/geocoder/geographies/addressbatch --output /content/drive/MyDrive/Classes/2022_Fall_DS501/geocoderesult_license.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218k  100  136k  100 84821   2352   1431  0:00:59  0:00:59 --:--:-- 29031


## ***Create new dataframe frome geocode results and prepare for convenience and performance in steps below.***

In [160]:
# path of batch tract information from census bureau api
tract = r'/content/drive/MyDrive/Classes/2022_Fall_DS501/geocoderesult_license.csv'

# create new dataframe of census tract info
df_tract = pd.read_csv(tract, header=None)

# report input vs output for geocoding
print('Total records input into batch geocoding API (total licenses): ', len(df_license))
print('Total unique locations of licenses: ', len(df_location))
print('Total records output from batch geocoding API: ', len(df_tract), '\n')

# create list of column names and add to df
cols = ['SiteID', 'address', 'GeocodeMatch', 'MatchType', 'address_caps', 'coordinates', 'tigerLineID', 'L-R', 'state', 'county', 'tract', 'last']
df_tract.columns = cols

# print number of missing tract records and then drop records with missing tract information
print('Total records with missing Census tract information: ', df_tract['tract'].isna().sum())
df_tract.drop(df_tract[df_tract['tract'].isna()].index, inplace=True)

# reset index of tract dataframe
df_tract.reset_index(inplace=True)

# cast tract column to integer to drop decimal
df_tract['tract'] = df_tract['tract'].astype('int64')

# cast tract column to string then create six digit string padded with zeros as necessary for standardized format
df_tract['tract'] = df_tract['tract'].astype('string').str.zfill(6)

# cast SiteID to string for proper merging in steps below
df_tract['SiteID'] = df_tract['SiteID'].astype('string')

# drop excess columns to simplify dataframe
df_tract.drop(['index', 'address_caps', 'L-R','state','county','last'], axis=1, inplace=True)

# print number of records in modified dataframe after dropping missing records
print('Total records with Census tract information: ', len(df_tract))

# print total number of unique tracts from geocoding
print('Total number of unique Cebsus tracts from geocoding: ', df_tract['tract'].nunique())

# display first 5 rows of dataframe
df_tract.head()


Total records input into batch geocoding API (total licenses):  1807
Total unique locations of licenses:  818
Total records output from batch geocoding API:  818 

Total records with missing Census tract information:  146
Total records with Census tract information:  672
Total number of unique Cebsus tracts from geocoding:  75


,SiteID,address,GeocodeMatch,MatchType,coordinates,tigerLineID,tract
0,488.0,"1116 Winburn Dr, Lexington, KY, 40511",Match,Exact,"-84.47593508899996,38.089491433000035",31945598.0,003804
1,9744.0,"2688 Pink Pigeon Pkwy, Lexington, KY, 40509",Match,Exact,"-84.42617792399994,38.02234922400004",31954114.0,003908
2,29912.0,"222 Rosemont Gdn, Lexington, KY, 40503",Match,Exact,"-84.52216845799995,38.019451135000054",31922565.0,002800
3,1121.0,"2300 Versailles Rd, Lexington, KY, 40504",Match,Exact,"-84.55210348399999,38.046713140000065",31951790.0,003000
4,15898.0,"205 S Limestone, Lexington, KY, 40508",Match,Non_Exact,"-84.49898297299995,38.04502051400004",31921805.0,000101


# ***Create new dataframe frome tract population JSON and prepare for convenience and performance in steps below.***

In [161]:
# path of population by tract information
pop = r'https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP05_0001E&for=tract:*&in=state:21&in=county:067'

# create new dataframe of tract population information
df_pop = pd.read_json(pop)

# use first record as column headers
df_pop.columns = df_pop.iloc[0]

# remove first row used for column headers
df_pop = df_pop.loc[1:]

# drop excess columns to simplify dataframe
df_pop.drop(['state', 'county'], axis=1, inplace=True)

# rename columns
df_pop.columns = ['name', 'population', 'tract']

# report total number of records (tracts)
print('Total number of tracts with population in Fayette County: ', len(df_pop))

# display population df
df_pop.head()

Total number of tracts with population in Fayette County:  82


,name,population,tract
1,"Census Tract 1.01, Fayette County, Kentucky",3171,000101
2,"Census Tract 1.02, Fayette County, Kentucky",1584,000102
3,"Census Tract 2, Fayette County, Kentucky",3696,000200
4,"Census Tract 3, Fayette County, Kentucky",2472,000300
5,"Census Tract 4, Fayette County, Kentucky",1983,000400


# ***Merge dataframes together to get tracts, population, liquor licenses, and locations in single dataframe.***

In [186]:
# join geocoded tract and population data into one dataframe using column tract
df_merge1 = pd.merge(df_tract, df_pop, on='tract')

# drop excess columns for readability
df_merge1.drop(['name','tigerLineID','coordinates','MatchType','GeocodeMatch'], axis=1, inplace=True)

# report total number of records in merged table
print('Total number of locations with tract and population data: ', len(df_merge1))

# display first 5 rows of merge dataframe
df_merge1.head()

Total number of locations with tract and population data:  672


,SiteID,address,tract,population
0,488.0,"1116 Winburn Dr, Lexington, KY, 40511",003804,6410
1,7356.0,"1511 Russell Cave Rd, Lexington, KY, 40505",003804,6410
2,28192.0,"413 W New Circle Rd, Lexington, KY, 40511",003804,6410
3,3771.0,"1515 Russell Cave Rd, Lexington, KY, 40505",003804,6410
4,8171.0,"1198 Winburn Dr, Lexington, KY, 40511",003804,6410


In [187]:
# create total dataframe with all data; join previously merged df with df_license using column SiteID
df_license_final = pd.merge(df_merge1, df_license, on='SiteID')

# drop excess columns for readability
df_license_final.drop(['County', 'City', 'PremisesStreet', 'PremisesCityState', 'Status', 'IssueDate', 'EffectiveDate', 'ExpiryDate', 'api_city', 'api_state', 'api_zip'], axis=1, inplace=True)

# report final df_license_final information
print('Total alcohol licenses in Fayette County: ', len(df_license))
print('Total alcohol licenses with tract information: ', len(df_license_final))

# display first 5 rows of dataframe
df_license_final.head()

Total alcohol licenses in Fayette County:  1807
Total alcohol licenses with tract information:  1479


,SiteID,address,tract,population,DBA,Licensee,LicenseNumber,LicenseType
0,488.0,"1116 Winburn Dr, Lexington, KY, 40511",003804,6410,WINBURN MINI MART 2,WINBURN MINI MART LLC,034-NQ-169896,NQ Retail Malt Beverage Package License
1,7356.0,"1511 Russell Cave Rd, Lexington, KY, 40505",003804,6410,Russell Cave Convenient Store,LOS HERMANOS MARKET LLC,034-NQ-192258,NQ Retail Malt Beverage Package License
2,28192.0,"413 W New Circle Rd, Lexington, KY, 40511",003804,6410,Tacos El Rey,"Tacos El Rey, LLC",034-RS-5759,Special Sunday Retail Drink License
3,28192.0,"413 W New Circle Rd, Lexington, KY, 40511",003804,6410,Tacos El Rey,"Tacos El Rey, LLC",034-NQ2-4020,NQ2 Retail Drink License
4,3771.0,"1515 Russell Cave Rd, Lexington, KY, 40505",003804,6410,Cowboy's,"KING-KELLY, INC.",034-NQ4-1068,NQ4 Retail Malt Beverage Drink License


In [188]:
# repeat steps above for dataframe with unique locations
df_location_final = pd.merge(df_merge1, df_location, on='SiteID')
df_location_final.drop(['County', 'City', 'PremisesStreet', 'PremisesCityState', 'Status', 'IssueDate', 'EffectiveDate', 'ExpiryDate', 'api_city', 'api_state', 'api_zip'], axis=1, inplace=True)

# report
print('Total alcohol licenses in Fayette County: ', len(df_location))
print('Total alcohol licenses with tract information: ', len(df_location_final))

# display
df_location_final.head()

Total alcohol licenses in Fayette County:  818
Total alcohol licenses with tract information:  672


,SiteID,address,tract,population,DBA,Licensee,LicenseNumber,LicenseType
0,488.0,"1116 Winburn Dr, Lexington, KY, 40511",003804,6410,WINBURN MINI MART 2,WINBURN MINI MART LLC,034-NQ-169896,NQ Retail Malt Beverage Package License
1,7356.0,"1511 Russell Cave Rd, Lexington, KY, 40505",003804,6410,Russell Cave Convenient Store,LOS HERMANOS MARKET LLC,034-NQ-192258,NQ Retail Malt Beverage Package License
2,28192.0,"413 W New Circle Rd, Lexington, KY, 40511",003804,6410,Tacos El Rey,"Tacos El Rey, LLC",034-RS-5759,Special Sunday Retail Drink License
3,3771.0,"1515 Russell Cave Rd, Lexington, KY, 40505",003804,6410,Cowboy's,"KING-KELLY, INC.",034-NQ4-1068,NQ4 Retail Malt Beverage Drink License
4,8171.0,"1198 Winburn Dr, Lexington, KY, 40511",003804,6410,Winburn Fruit Super Mercado,WINBURN FRUIT SUPER MARKET LLC,034-NQ-7586,NQ Retail Malt Beverage Package License


# ***Analyze data using the alochol license per capita and alcohol unique locations per capita.***

In [189]:
# get total count of liquor license locations in each tract
df_license_final['llcount_all'] = df_license_final.groupby(['tract'])['tract'].transform('count')

# sort dataframe by liquor license per tract in descending order
df_license_final.sort_values(by='llcount_all', ascending=False, inplace=True)

# reset index
df_license_final.reset_index(inplace=True)

# display first 5 rows of dataframe
df_license_final.head()

,index,SiteID,address,tract,population,DBA,Licensee,LicenseNumber,LicenseType,llcount_all
0,319,4834.0,"200 W Main St, Lexington, KY, 40507",000101,3171,Harveys,"HARVEY'S BAR, LLC",034-NQ4-1679,NQ4 Retail Malt Beverage Drink License,196
1,226,6577.0,"307 W Short St, Lexington, KY, 40507",000101,3171,Postmaster's Pub,"3 Behrs, Inc.",034-SB-1635,Supplemental Bar License,196
2,224,2632.0,"214 E Main St, Lexington, KY, 40507",000101,3171,Kentucky Theatre,"FRIENDS OF KENTUCKY THEATRE, INC.",034-LD-188055,Quota Retail Drink License,196
3,223,2632.0,"214 E Main St, Lexington, KY, 40507",000101,3171,Kentucky Theatre,"FRIENDS OF KENTUCKY THEATRE, INC.",034-NQ4-188054,NQ4 Retail Malt Beverage Drink License,196
4,222,18821.0,"152-154 Market St, Lexington, KY, 40507",000101,3171,Belle's Bar,Belle's Bar LLC,034-NQ4-2436,NQ4 Retail Malt Beverage Drink License,196


In [190]:
# repeat steps above for dataframe with unique locations instead of ALL liquor licenses
# get total count of liquor license locations in each tract
df_location_final['llcount_unique'] = df_location_final.groupby(['tract'])['tract'].transform('count')

# sort dataframe by liquor license per tract in descending order
df_location_final.sort_values(by='llcount_unique', ascending=False, inplace=True)

# reset index
df_location_final.reset_index(inplace=True)

# display first 5 rows of dataframe
df_location_final.head()


,index,SiteID,address,tract,population,DBA,Licensee,LicenseNumber,LicenseType,llcount_unique
0,110,14719.0,"310 W Short St, Lexington, KY, 40507",000101,3171,Creaux,"Holiday, LLC",034-LD-2105,Quota Retail Drink License,66
1,112,29261.0,"102 W Vine St, Lexington, KY, 40507",000101,3171,Ethereal Brewing Public House,"The Inferno, LLC",034-LD-160908,Quota Retail Drink License,66
2,82,4381.0,"266 E Second St, Lexington, KY, 40508",000101,3171,Bluegrass Progressive Club Inc,"THE BLUEGRASS PROGRESSIVE CLUB, INC.",034-NQ3-1121,NQ3 Retail Drink License,66
3,81,21570.0,"112 N Mill St, Lexington, KY, 40507",000101,3171,KM Food Mart,Keola Inc,034-NQ-6026,NQ Retail Malt Beverage Package License,66
4,80,15367.0,"123 Cheapside, Lexington, KY, 40507",000101,3171,Wild Cat Saloon Lex,WCS LEX Inc,034-NQ4-1993,NQ4 Retail Malt Beverage Drink License,66


In [191]:
# calculate liquor licenses per capita for each tract and put into new column for all licenses; cast population to integer data type for calculation 
df_license_final['ll_per_capita_all'] = df_license_final['llcount_all'] / df_license_final['population'].astype('int64')

# repeat above for unique locations 
df_location_final['ll_per_capita_unique'] = df_location_final['llcount_unique'] / df_location_final['population'].astype('int64') 


# calculate liquor licenses per 1000 people for each tract; multiply columns calculated above by 1000
df_license_final['ll_per_1000_all'] = df_license_final['ll_per_capita_all'] * 1000
df_location_final['ll_per_1000_unique'] = df_location_final['ll_per_capita_unique'] * 1000

## ***Present results of rates of alcohol availability versus total number of alcohol licenses per neighborhood (Census tract).***

In [214]:
# top 20 rates of alcohol availability by number of licenses; grouping by tract; calculate mode for each tract
top20_license = df_license_final.groupby('tract')['ll_per_1000_all'].agg(pd.Series.mode)

# display top 20 highest rates of alcohol availability by all licenses
top20_license.head(20)

tract
000101    61.810155
000102    32.196970
000200    12.445887
000300     8.090615
000400     3.025719
000500     9.381413
000600     3.064128
000700    17.624521
000801     0.768344
000802     1.336643
000900    14.955470
001000    42.553191
001100     3.162412
001300     7.977475
001400     5.790646
001500     1.727116
001600    10.766581
001800     6.191950
001900     2.593258
002001     1.779072
Name: ll_per_1000_all, dtype: float64

In [215]:
# top 20 rates of alcohol availability by unique locations; grouping by tract; calculate mode for each tract
top20_location = df_location_final.groupby('tract')['ll_per_1000_unique'].agg(pd.Series.mode)

# display top 20 highest rates of alcohol availability by unique locations
top20_location.head(20)

tract
000101    20.813623
000102    10.732323
000200     4.329004
000300     4.449838
000400     2.521432
000500     3.224861
000600     1.313198
000700     7.662835
000801     0.384172
000802     0.572847
000900     5.041170
001000    18.147685
001100     1.807093
001300     4.692633
001400     3.118040
001500     1.727116
001600     5.167959
001800     3.095975
001900     1.396369
002001     1.132137
Name: ll_per_1000_unique, dtype: float64

In [217]:
# checking if both datasets produce same results of rates of alcohol availability
top20_license.index == top20_location.index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

***The data above show that the top 20 neighborhoods with the highest number of alcohol licenses per 1000 people is the same as the number of locations serving alcohol per 1000 people in neighborhoods of Fayette County, which should be expected. The neighborhood with the highest rate of alcohol availability is tract 000101 with almost, which is the downtown area of Lexington and should also be expected. In this tract, there are almost 21 locations per 1000 people, and is almost double the next largest rate of alcohol availability (~11 locations per 1000 people).***

In [218]:
# top 20 tracts with highest number of licenses; grouping by tract; calculate mode for each tract
top20_license_loc = df_license_final.groupby('tract')['llcount_all'].agg(pd.Series.mode)

# display top 20 highest number of licenses
top20_license_loc.head(20)

tract
000101    196
000102     51
000200     46
000300     20
000400      6
000500     32
000600     14
000700     46
000801      2
000802      7
000900     89
001000     68
001100     14
001300     17
001400     13
001500      4
001600     25
001800     18
001900     13
002001     11
Name: llcount_all, dtype: int64

In [219]:
# top 20 tracts with highest number of alcohol serving locations; grouping by tract; calculate mode for each tract
top20_location_loc = df_location_final.groupby('tract')['llcount_unique'].agg(pd.Series.mode)

# display top 20 highest number of alcohol serving locations
top20_location_loc.head(20)

tract
000101    66
000102    17
000200    16
000300    11
000400     5
000500    11
000600     6
000700    20
000801     1
000802     3
000900    30
001000    29
001100     8
001300    10
001400     7
001500     4
001600    12
001800     9
001900     7
002001     7
Name: llcount_unique, dtype: int64

In [220]:
# checking if both datasets produce same results of highest number of licenses vs locations
top20_license_loc.index == top20_location_loc.index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

***The data above show that the top 20 neighborhoods with the highest number of alcohol licenses is the same as the number of locations serving alcohol in neighborhoods of Fayette County, which should be expected. The neighborhood with the highest number of licenses and number of alcohol-serving locations is Census Tract 000101, which is the downtown Lexington area.***

In [221]:
# compare alcohol rates vs license counts
top20_license_loc.index == top20_license.index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [222]:
# compare alcohol rates vs license counts
top20_location_loc.index == top20_location.index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

***Finally, the data above show that the top 20 neighborhoods with the highest rates of alcohol availability (according to both number of licenses and unique locations per capita) is the same as the top 20 neighborhoods with the highest number of alcohol licenses and alcohol-serving locations.***